In [1]:
import os
import datetime
import sqlite3
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

In [3]:
# Ubicaciones de trabajo
FOLDER = os.getcwd()
DATA_FOLDER = os.path.join(FOLDER, "data")

# Carga de variables de entorno
load_dotenv(os.path.join(FOLDER, ".env"))

True

In [3]:
# Lectura de los datos de los CUS simulados
cus_data = pd.read_excel(
    os.path.join(DATA_FOLDER, "data_cus.xlsx"),
    sheet_name=0,
    dtype=str,
)

In [4]:
# Conexion con OpenAI
model_gen = AzureChatOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_GPT4OMINI_DEPLOY"),
    temperature=0
)

In [5]:
# Funcion para extraer informacion de la base de datos
@tool
def get_transaction(user_id, db=cus_data):

    """
    Use this functon to find and return a table with the user's transactions
    """

    # Basicamente lo que hacemos es filtrar la base de datos de las transacciones 
    # Unicamente para las del usuario que estoy buscando, nada más
    # El resultado es una TABLA REDUCIDA que el LLM luego va leer para encontrar lo que le pidamos

    return db[db["cedula"]==user_id]

In [6]:
# Función para generar un numero consecutivo que pueda meter en la base de datos
@tool
def generate_case_number():

    """
    Use this function to generate a case number for the user's case.
    """

    conn = sqlite3.connect(
        os.path.join(FOLDER, "case_db.db")
    )
    cursor = conn.cursor()

    df = pd.read_sql_query("SELECT * FROM casos", conn)
    number = df["case_id"].tolist()[-1]
    number += 1

    return number


In [101]:
# Funcion para enviar correo al usuario
@tool
def send_user_email(user_email, user_name, case_info, case_number, date):

    """
    Use this function to send emails to the user as soon as you have the user's email.
    """
    
    # Credenciales
    sender_email = "anavi.bbva@gmail.com"
    sender_password = "nmmi duca pbqv etak"

    # Contenido
    subject = f"ANAVI (BBVA) - CASO #{case_number}"
    body = f"""
¡Hola! {user_name}
Soy ANAVI - Asistente Virtual de BBVA

He recolectado la siguiente información sobre tu caso:

Fecha: {date}
Número de caso: {case_number}
Descripción: {case_info}

Le he asignado tu caso al área encargada y será atendido a la brevedad posible. Tendrás tu respuesta \
en un máximo de 5 días hábiles. Recuerda que todos nuestros canales estan disponibles para ti.
"""

# Envio
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = user_email
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))

    # Creating the server connection and sending the email
    try:
        # Setting up the SMTP server (Gmail's SMTP server)
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Upgrade the connection to secure

        # Logging into the email account
        server.login(sender_email, sender_password)

        # Sending the email
        server.sendmail(sender_email, user_email, message.as_string())

        # Closing the server connection
        server.quit()

        print("Correo enviado exitosamente al usuario")

    except Exception as e:
        print(f"Error enviando correo del usuario: {e}")


# Funcion para enviar el correo al área encargada
@tool
def send_area_email(user_name, user_document, area_name, case_info, case_number, date):

    """
    Use this function to send emails to the responsible area as soon as you have the area's name and all 
    the info of the user's case.
    """
    
    # Utilizamos el nombre del area que detecto el bicho para relacionarlo con el correo
    # Por el momento es mas confiable a permitir que el bicho lo haga por si mismo
    # con un modelo mejor, resultado mejor y nos ahorramos esta parte
    # o quizas no...?
    mails = {
        "TRANS": "juandavid.duran@bbva.com",
        "SUCUR": "leslykatherine.pineros@bbva.com",
        "SEGUR": "lauraconsuelo.caro@bbva.com",
        "PRODU": "santiago.moreno.rodriguiez@bbva.com"
    }
    area_email = mails[area_name]

    names = {
        "TRANS": "Transacciones",
        "SUCUR": "Sucursales Físicas",
        "SEGUR": "Seguros",
        "PRODU": "Productos Financieros"
    }
    area_name = names[area_name]

    # Credenciales
    sender_email = "anavi.bbva@gmail.com"
    sender_password = "nmmi duca pbqv etak"

    # Contenido
    subject = f"ANAVI (BBVA) - CASO #{case_number}"
    body = f"""
He recolectado el siguiente caso:

Fecha: {date}
Nombre: {user_name}
Documento: {user_document}
Número de caso: {case_number}
Descripción: {case_info}
Area: {area_name}

He identificado que este caso pertenece al área de {area_name} y como consecuencia estas recibiendo el aviso.
Toda la información ya ha sido documentada y almacenada en la bae de datos correspondiente.
"""

# Envio
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = area_email
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))

    # Creating the server connection and sending the email
    try:
        # Setting up the SMTP server (Gmail's SMTP server)
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Upgrade the connection to secure

        # Logging into the email account
        server.login(sender_email, sender_password)

        # Sending the email
        server.sendmail(sender_email, area_email, message.as_string())

        # Closing the server connection
        server.quit()

        print("Correo enviado exitosamente al area encargada")

    except Exception as e:
        print(f"Error enviando correo del area: {e}")

# Prueba
send_user_email.invoke({"user_email": "diegoalejandro.peralta@bbva.com", "user_name": "Diego", "case_info": "prueba", "case_number": "0000", "date": "2024"})

Correo enviado exitosamente al usuario


In [8]:
# Función para generar un comprobante
@tool
def generate_file(user_id, date, description, value, cus, account, state):
    
    """
    Use this function to use the transaction's info to create a image that 
    can be donwloaded by the user.
    """

    data = {
        "Cédula": user_id,
        "Fecha": date,
        "Descripción": description,
        "Valor": value,
        "CUS": cus,
        "Estado": account,
        "Cuenta de Ahorros": state
    }

    # Acá dejamos los parametros basicos de la libreria para generar imagenes
    font = "arial.ttf"
    boldfont = ImageFont.truetype(font, 20)
    normalfont = ImageFont.truetype(font, 20)

    # Esta es la base en blanco para dibujar por decirlo asi
    # Este tamaño es el que ha funcionado que no se ve feo
    img = Image.new("RGB", (800, 600), color=(255, 255, 255))
    d = ImageDraw.Draw(img)
    # Este es el logo fondo blanco del BBVA para que se vea coqueto
    logo = Image.open("logo.jpg")
    logo = logo.resize((200, 90))
    img.paste(logo, ((800 - logo.width) // 2, 20))

    # Mini función para centrar el texto que se vea mejor
    def draw_centered_text(draw, text, y, font, image_width, fill_color):
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        x = (image_width - text_width) // 2
        draw.text((x, y), text, font=font, fill=fill_color)

    # Si el estado es exitoso entonces sera de color de verde y si no, es rojo pues
    color = (0, 128, 0) if data['Estado'] == "Exitoso" else (255, 0, 0)

    # Agregar texto centrado a la imagen con descripciones en negrita y valores en normal
    y_offset = 130
    draw_centered_text(d, f"Pago {data['Estado'].lower()}", y_offset, boldfont, img.width, color)
    y_offset += 40
    draw_centered_text(d, f"Valor: {data['Valor']}", y_offset, normalfont, img.width, (0, 0, 0))
    y_offset += 40
    draw_centered_text(d, f"Fecha: {data['Fecha']}", y_offset, normalfont, img.width, (0, 0, 0))
    y_offset += 40
    draw_centered_text(d, f"Producto o servicio: {data['Descripción']}", y_offset, normalfont, img.width, (0, 0, 0))
    y_offset += 40
    draw_centered_text(d, f"Cédula: {data['Cédula']}", y_offset, normalfont, img.width, (0, 0, 0))
    y_offset += 40
    draw_centered_text(d, f"Cuenta de Ahorros: {data['Cuenta de Ahorros']}", y_offset, normalfont, img.width, (0, 0, 0))
    y_offset += 40
    draw_centered_text(d, f"Código de confirmación (CUS): {data['CUS']}", y_offset, normalfont, img.width, (0, 0, 0))

    # Guardar la imagen
    img.save(f"{cus}.png")

    print("Imagen generada exitosamente.")

    return "Imagen generada"

In [9]:
# Función para obtener la fecha y hora del caso
@tool
def get_case_date():

    """
    Use this function to get the current date and time to use in the user's case.
    """

    return  datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
# Funcion para almacenar la informacion
@tool
def store_case(case_id, date, user_name, user_id, user_email, case_info):

    """
    Use thus function to store in a database the user's case info as sson as you have 
    all the information.
    """

    conn = sqlite3.connect(
        os.path.join(FOLDER, "case_db.db")
    )
    cursor = conn.cursor()

    # No tiene sentido que agreguemos un caso que ya existe
    # Pero si tiene sentido que me alerte cuando eso sucede
    cursor.execute('''
    SELECT * FROM casos WHERE case_id = ?
    ''', (case_id,))
    existing_case = cursor.fetchone()

    if existing_case:
        print("Ya existe ese caso")
        response = "El caso que intentas agregar ya existe en la BD."
    else:
        cursor.execute('''
        INSERT INTO casos (case_id, date, user_name, user_id, user_email, case_info)
        VALUES (?, ?, ?, ?, ?, ?)
        ''', (case_id, date, user_name, user_id, user_email, case_info))
        conn.commit()
        print("Caso agregado a la bd")
        response = "El caso ha sido agregado exitosamente a la BD"

    return response

In [4]:
conn = sqlite3.connect(
        os.path.join(FOLDER, "case_db.db")
    )
cursor = conn.cursor()

df = pd.read_sql_query("SELECT * FROM casos", conn)

df

,case_id,date,user_name,user_id,user_email,case_info
0,374647374,2025-02-04 14:52,Juan Pérez,101,juan.perez@example.com,Información detallada del caso.
1,374647375,2025-02-04 17:45:20,Napoleon,1117773333,diegoalejandro.peralta@bbva.com,Problema con retiro en cajero automático.
2,374647376,2025-02-06 10:41:17,Juan,2345656,juandavid.duran@bbva.com,Problema con retiro en cajero automático. Inte...


In [13]:
# Lista de herramientas
tools = [send_user_email, send_area_email, generate_case_number, get_transaction, generate_file, store_case, get_case_date]

In [25]:
# Diseño de prompt
template = """
<CONTEXTO>>
Eres una IA llamada ANAVI.
Trabajas para el banco BBVA.
Tu objetivo es ayudar a los usuarios a resolver su problemas.

<<CONVERSACION>>
Cuando saludes al usuario debes introducirte por tu nombre y tu función y dile \
al usuario que estas preparada para ayudarlo.
Antes de preguntarle cualquier cosa al usuario debes solicitarle su nombre para \
poder dirigirte a él de forma adecuada.

<<PERSONALIDAD>>
Debes ser amable y respetuosa en todo momento.
Debes ser muy clara al momenot de hablar como si trataras con un niño de 10 años.
No olvides despedirte amablemente del usuario cuando este se despida primero.

<<IMPORTANTE>>
Antes de solicitarle al usuario cualquier información personal como la cédula, el documento, el número de cuenta o el correo electrónico, \
debes preguntarle si autoriza el tratamiento de sus datos conforme a la Autorización de Tratamiento de Datos Personales disponible en el siguiente enlace:
https://www.bbva.com.co/content/dam/public-web/colombia/documents/home/prefooter/politicas-informacion/DO-03-Autorizacion-tratamiento-datos-personales.pdf.
Si el usuario no está de acuerdo, debes informarle que no puedes continuar con el proceso.
Si el usuario insiste en no aceptar las condiciones debes informar que no puedes recolectar datos y finalizar la conversación.

<METODO DE TRABAJO>>
Una vez que tienes el nombre del usuario debes preguntarle cuál es el motivo de su consulta, o en que lo puedes ayudar.
Estos son los casos que estas preparada para resolver:

- TRANSACCIONES
Cuando el usuario te pregunte que necesita averiguar sobre una transacción que no ve reflejada, que nunca se realizó, que falló o \
simplemente recordar la fecha, el valor o sobre que trataba una transacción deberás hacer lo siguiente:
1. Solicitar el número de documento del usuario y los ultimos 4 digitos de su cuenta de ahorros
Cuando tengas el número de documento, debes utilizarlo para buscar sus transacciones.
Una vez obtengas las transacciones debes validar si los ultimos 4 digitos de la cuenta coinciden con los que te dio el usuario.
Si todo esta en orden, debes buscar las que estan relacionadas con la petición del usuario.
2. Debes responder al usuario con la lista de las transacciones que encajan con la descripción del usuario y \
debes preguntarle al usuario si alguna de esas transacciones es la que estab buscando.
3. Debes preguntarle al usuario si desea el comprobante de la transacción que ha elegido. Entonces, debes responderle con \
la información de la transacción y decirle que puede descargarla.

- PROBLEMAS CON ACTIVACION DE PRODUCTOS COMO TARJETAS
Cuando el usuario te hable sobre un problema con la activación de productos debes informarle que vas a gestionar su caso.
En este caso no es necesario que solicites la información de la cuenta de ahorros.

- PROBLEMAS CON RETIROS Y CONSIGNACIONES EN SUCURSALES FISICAS, OFICINAS O CAJEROS
Cuando el usuario te hable sobre un problema cuando esta realizando retiros o consignaciones debes informarle que vas a gestionar su caso.
En este caso no es necesario que solicites la información de la cuenta de ahorros.

- PROBLEMAS PARA ADQUIRIR O CANCELAR SEGUROS
Cuando el usuario te hable sobre un problema con los seguros debes informarle que vas a gestionar su caso.
En este caso no es necesario que solicites la información de la cuenta de ahorros.

<<GESTION DE CASOS>>
Para obtener la fecha y hora del caso debes utilizar la función correspondiente.
Para obtener el número del caso debes obtener la función correspondiente.
En todos los tipos de casos debes ayudar al usuario gestionar los casos. En el caso de las transacciones, tienes las herramientas \
para ayudar al usuario. Si la solución no es posible y para los demás casos debes ayudar a gestionar su caso.
En ese caso, necesitas el correo del usuario para poder enviar la información del caso al área requerida.
Una vez tengas el correo del usuario debes:
1. Enviar el correo al usuario con la información de su caso.
2. Enviar el correo al area encargada con el caso del usuario.

<<CLASIFICACION DE CASOS>>
Cuando termine la conversación deberas enviar la información al área encargada.
Dependiendo del caso que te solicite el usuario cuando vayas a enviar el correo al area encargada debes usar los siguientes nombres:
- Problemas de activación de productos (tarjetas, cuentas o seguros): PRODU
- Información y/o consulta sobre transacciones: TRANS
- Problemas con retiros y/o consignación de dinero en sucursales físicas, oficinas o cajeros automaticos: SUCUR
- Problemas sobre la adquisición y/o cancelación de seguros: SEGUR
"""

In [26]:
# Memoria
memory = MemorySaver()

In [40]:
# Create the message with the custom system prompt
messages = [
    SystemMessage(content=template),
    HumanMessage(content="mi correo es diegoalejandro.peralta@bbva.com")
]

# Your existing code
agent_executor = create_react_agent(model_gen, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "00022"}}
response = agent_executor.invoke({"messages": messages}, config)

c:\Users\O014796\AppData\Local\miniconda3\envs\anavi\Lib\site-packages\pydantic\json_schema.py:2279: PydanticJsonSchemaWarning: Default value         cedula                fecha                               descripcion  \
0   1112223334  31/01/2025 08:23 AM                    Compra en Tienda Éxito   
1   1112223334  30/01/2025 09:15 AM                  Pago de servicio de agua   
2   1112223334  29/01/2025 10:40 AM               Cena en Andrés Carne de Res   
3   1112223334  28/01/2025 11:22 AM          Pago de servicio de electricidad   
4   1112223334  27/01/2025 02:12 PM           Compra en Supermercado Olímpica   
5   1112223334  26/01/2025 03:50 PM                   Pago de servicio de gas   
6   1112223334  25/01/2025 05:30 PM         Almuerzo en Restaurante El Corral   
7   1112223334  24/01/2025 06:40 AM                    Compra en Tienda Jumbo   
8   1112223334  23/01/2025 07:50 AM              Pago de servicio de teléfono   
9   1112223334  22/01/2025 08:30 AM             

Caso agregado a la bd
Correo enviado exitosamente al area encargada


In [41]:
print(response["messages"][-1].content)

He gestionado tu caso sobre el problema con el retiro en el cajero automático. La información ha sido enviada al área correspondiente y ellos se encargarán de investigar lo sucedido.

Si tienes alguna otra pregunta o necesitas más ayuda, no dudes en decírmelo. ¡Estoy aquí para ayudarte!


In [98]:
response["messages"][-1].content

'He gestionado tu caso sobre el problema con el retiro en el cajero automático. La información ha sido enviada al área correspondiente y ellos se encargarán de investigar lo sucedido.\n\nSi tienes alguna otra pregunta o necesitas más ayuda, no dudes en decírmelo. ¡Estoy aquí para ayudarte!'

In [96]:
import numpy as np
str(np.random.randint(1, 500)).zfill(5)

'00018'